In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict
from pprint import pprint
import datetime
import re
from IPython.display import clear_output

In [10]:
INPUT_FILE_ENG = "../1 preproc/data_preproc/data_subset_eng_5000_from_EW-SEW_08_24_18h"
INPUT_FILE_FRE = "../1 preproc/data_preproc/data_subset_fre_5000_from_WiViCo2_08_31_10h"
INPUT_FILE_GER = "../1 preproc/data_preproc/data_subset_ger_5000_from_DEPlain_09_11_12h"

######################
LANGUAGE = 'eng'
MODEL = 'ChatGPT'
SOURCE_DATASET = 'EW-SEW'
SIZE_OF_OUTPUT_DATASET = 1000+100
######################

output_file_name =  "out_" + LANGUAGE + "_from_" + SOURCE_DATASET

print("DOUBLE-CHECK THIS:")
print(output_file_name)

DOUBLE-CHECK THIS:
out_eng_from_EW-SEW


In [15]:
print("LANGUAGE IS: ###", LANGUAGE)
if LANGUAGE == 'fre':
    input_file = INPUT_FILE_FRE
elif LANGUAGE == 'ger':
    input_file = INPUT_FILE_GER
elif LANGUAGE == 'eng':
    input_file = INPUT_FILE_ENG
print("input_file is: ", input_file, "\n____\n")


with open(input_file) as file:
    lines = [line.strip() for line in file.readlines() if line.strip() != ""]

    for line in lines[:3]:
        print(line)

tuples_list = [(line.split("|")[0], line.split("|")[1].strip()) for line in lines]

df = pd.DataFrame(columns=["source_index", "original_sent", "simplified_sent"])
print(df)

pprint(tuples_list[:5])

LANGUAGE IS: ### eng
input_file is:  ../1 preproc/data_preproc/data_subset_eng_5000_from_EW-SEW_08_24_18h 
____

57224|Toxicofera (Greek for "those who bear toxins") , is a hypothetical clade which represents about 4600 species (nearly 60 %) of extant squamates (scaled lizards.)
117533|The islands of the Caribbean Sea, collectively known as the West Indies, are sorted by size and location into the Bahamas (or Lucayan archipelago, which includes the Turks and Caicos Islands) , the Lesser Antilles, and the Greater Antilles.
3709|Some websites do not allow typographic quotation marks or apostrophes in posts (one such example being YouTube) .
Empty DataFrame
Columns: [source_index, original_sent, simplified_sent]
Index: []
[('57224',
  'Toxicofera (Greek for "those who bear toxins") , is a hypothetical clade '
  'which represents about 4600 species (nearly 60 %) of extant squamates '
  '(scaled lizards.)'),
 ('117533',
  'The islands of the Caribbean Sea, collectively known as the West Ind

In [16]:
prompts_eng = [
    "Simplify this sentence: ",
    "Simplify this sentence, return only the simplified sentence: ",
    "Simplify the following new complex sentence. Here's an example: Complex: Hitherto, smoking was prohibited in the facilities. Simple: Smoking was forbidden here until now. New complex sentence: ",
    "Rewrite the following sentence to be as simple and concise as possible, while keeping all the essential information intact: ",
    "Make this sentence easier to understand: ",
    "Simplify this original sentence. The result should be a sentence with a easy sentence structure and should include only easy words. Original sentence: ",
    "Simplify this original sentence by creating an alternative version with a simpler sentence structure and fewer difficult words. Original sentence: ",
    "Make an alternative version of this original sentence which is shorter, exhibits a simpler structure and easier words.  For example, \"Hitherto, smoking was prohibited in the facilities.\"  could become \"Smoking was forbidden here until now.\". Original sentence: ",
    "Explain the following to a five year old: ",
    "Simplify this original sentence. Change it as much as possible without altering the meaning. Return only the simplified sentence. Original Sentence: ",
    """Simplify each of these 50 unrelated sentences one by one. Return only the simplified sentences, no numbering, no comments or explanations.
ORIGINAL SENTENCES : \n\n""",
    """Turn each of these 50 unrelated sentences into simple language, while retaining all of the most important information from the original. Simple language is characterized by simple sentence structures and simple words. Answer by giving only the simplified sentences, no numbering, no other remarks.
ORIGINAL SENTENCES : \n\n"""
]

prompts_ger = [
    """Verwandle jeden dieser 50 nicht zusammenhängenden Originalsätze einzeln in einfache Sprache. Behalte dabei die wichtigsten Informationen aus dem jeweiligen Originalsatz bei. Einfache Sprache ist durch einfache Satzstrukturen und leichte Wörter gekennzeichnet. Antworte, indem du nur die vereinfachten Sätze auflistest, ohne Nummerierung und ohne sonstige Hinweise.
ORIGINALSÄTZE : \n\n"""

]

prompts_fre = [
    """Convertis chacune de ces 50 phrases originales non liées en langage simple, une par une. Conserve les informations les plus importantes de la phrase originale. Le langage simple se caractérise par des structures de phrases simples et l'utilisage des mots faciles. Réponds en énumérant uniquement les phrases simplifiées, sans numérotation ni autres remarques.
PHRASES ORIGINALES : \n\n"""

]

if LANGUAGE == 'fre':
    prompts = prompts_fre
elif LANGUAGE == 'ger':
    prompts = prompts_ger
elif LANGUAGE == 'eng':
    prompts = prompts_eng

instruction_prompt_used = prompts[-1]
print(instruction_prompt_used)

Turn each of these 50 unrelated sentences into simple language, while retaining all of the most important information from the original. Simple language is characterized by simple sentence structures and simple words. Answer by giving only the simplified sentences, no numbering, no other remarks.
ORIGINAL SENTENCES : 




In [17]:
def process_text_into_list_of_sents(input_text):
    # Split the input by line breaks
    
    if '\n' in input_text:
        sentences = input_text.splitlines()
    else:
        sentences = re.split(r'(?<!\b\w\.\w)(?<!\d\.\d)(?<![A-Z]\.)(?<=\.|\?|\!)\s', input_text)

    # Initialize a list to hold the cleaned sentences
    cleaned_sentences = []

    for sentence in sentences:
        # Remove leading/trailing whitespaces
        sentence = sentence.strip()

        # Remove bullet points, numbers, and any special characters
        sentence = re.sub(r'^[\d\.\-\•\•]+\s*', '', sentence)  # Matches numbers, bullets, etc.

        # Only append non-empty sentences
        if sentence:
            cleaned_sentences.append(sentence)

    return cleaned_sentences

In [18]:
def batchwise_inference(input_list, dataframe, batch_size=50):

    output_list = []  # Initialize the output list

    # Iterate over input_list in chunks of batch_size
    for i in range(0, len(input_list), batch_size):

        batch_number = round(i/batch_size)

        if i >= SIZE_OF_OUTPUT_DATASET:
            print("i is ", i, "--> Size of output dataset reached. Execution is finished.")
            break

        #clear_output(wait=True)            

        # Take a batch of 50 elements
        input_batch = input_list[i:i + batch_size]
        output_batch = []

        #print("______ INPUT batch ", str(batch_number), " ______")
        print(instruction_prompt_used)
        # 1. Print each element in the batch separated by line breaks
        for element in input_batch:
            print(element[-1]+"\n")
        
        # 2. Print separator "_____"
        print("_____\n\n")

        while True:

            # 3. Wait for user input
            model_output = input(str("Enter input for batch number, " + str(batch_number) +": "))
            # 4. Call my_function with user input and extend the result to output_list
            processed_output = process_text_into_list_of_sents(model_output)


            if len(processed_output) != batch_size:
                print("Input invalid. Enter input correctly.")
                print("Len of proc'ed output: ", len(processed_output))
                for s in processed_output:
                    print(s, "\n")
            else:
                # If result is valid, break the loop and extend the output list
                output_list.extend(processed_output)
                output_batch.extend(processed_output)
                break
        

        print("______MODEL OUTPUT:______")
        for sent in processed_output:
            print(sent, "\n")

            
        for i, elem in enumerate(input_batch):
            triple=[elem[0], elem[1], output_batch[i]]
            dataframe.loc[len(dataframe)] = triple

    return output_list



In [19]:
def batchwise_inference_simple(input_list, dataframe,  batch_size=50):
    # Iterate over input_list in chunks of batch_size
    for i in range(0, len(input_list), batch_size):

        clear_output(wait=True)

        batch_number = round(i/batch_size)

        if i >= SIZE_OF_OUTPUT_DATASET:
            print("i is ", i, "--> Size of output dataset reached. Execution is finished.")
            break

        input_batch = input_list[i:i + batch_size]

        print(instruction_prompt_used)

        _ = input(str("Press enter to get sentences"))
        clear_output(wait=True)

        for element in input_batch:
            print(element[-1]+"\n")


    return None
        
#batchwise_inference_simple(tuples_list, df)

In [ ]:
for element in tuples_list[:SIZE_OF_OUTPUT_DATASET]:
            print(element[0]+"\n")

In [21]:
print(instruction_prompt_used)

Turn each of these 50 unrelated sentences into simple language, while retaining all of the most important information from the original. Simple language is characterized by simple sentence structures and simple words. Answer by giving only the simplified sentences, no numbering, no other remarks.
ORIGINAL SENTENCES : 




In [26]:
current_index = 'reset'

In [49]:
if 'current_index' not in globals() or current_index == "reset":
    current_index = 0  # This will be reset to 0 when the code runs for the first time

# Define the batch size (50 elements at a time)
batch_size = 50

# Check if the current_index exceeds z (the length of the list or the end limit)
if current_index >= SIZE_OF_OUTPUT_DATASET:
    print("Reached the end of the list. No more elements to display.")
else:
    # Get the elements for the current batch and print them

    print(instruction_prompt_used)

    for element in tuples_list[current_index:current_index + batch_size]:
        print(element[-1] + "\n")
    
    # Update the current index for the next batch
    current_index += batch_size

Reached the end of the list. No more elements to display.


In [18]:
df.loc[1002]

IndexError: single positional indexer is out-of-bounds

Saving the dataset (0/1 shards):   0%|          | 0/1150 [00:00<?, ? examples/s]